# 라즈베리파이 실시간 전압/전류 측정

## ⚙ 구성
|항목|모델명|용도|
|-------|-------|-------|
전압 측정|ZMPT101B|220V 전압 감지(출력: 아날로그)
전류 측정|ACS712 (5A/20A/30A)|부하 전류 감지 (출력: 아날로그)
ADC|ADS1115|아날로그 센서 → 디지털 신호로 변환
저장 형식|CSV|전처리 용이, 속도 빠름
측정 주기|100Hz(10ms 간격, 일반 논문에서 자주 사용)

## 🧠 흐름
- ZMPT101B & ACS712 (Analog)
- ADS1115 (I2C → Raspberry Pi)
- 실시간 측정 (0.01초 간격)
- CSV 파일로 저장

## ⚠️ 참고사항
- ACS712 전류 변환 공식: `current (A) = (measured_voltage - Vcc/2) / sensitivity` (예: 20A 모델은 100mV/A, Vcc=5V일 때 2.5V 중심 기준)
- ZMPT101B도 보정이 필요하지만, 일단 상대적 변화 감지용으로도 사용 가능

In [10]:
import time
import board
import busio
import adafruit_ads1x15.ads1115 as ADS
from adafruit_ads1x15.analog_in import AnalogIn
import pandas as pd
import numpy as np

In [11]:
# I2C 및 ADC 설정
i2c = busio.I2C(board.SCL, board.SDA)
ads = ADS.ADS1115(i2c)
ads.gain = 1  # 1: ±4.096V 범위 (ZMPT, ACS712에 적절)

# 센서 채널 지정
voltage_ch = AnalogIn(ads, ADS.P0)
current_ch = AnalogIn(ads, ADS.P1)

# # 보정 파라미터 (초기값)
# ZMPT_offset = 2.510
# ZMPT_scale = 941.68

# ACS712_offset = 2.552
# ACS712_sensitivity = 0.0925

# 센서 기준 오프셋
ZMPT_offset = 2.554
ZMPT_scale = 1000.0     # → 나중에 부하 걸고 조정 예정

ACS712_offset = 2.508
ACS712_sensitivity = 0.1  # 또는 0.05

In [14]:
data = []
count = 0

try:
    while True:
        now = time.time()
        label = 0  # 기본은 정상

        try:
            # 센서 원시 전압값 (정상)
            v_raw = voltage_ch.voltage
            c_raw = current_ch.voltage

            # 보정 계산
            real_voltage = (v_raw - ZMPT_offset) * ZMPT_scale
            real_current = (c_raw - ACS712_offset) / ACS712_sensitivity

        except OSError as e:
            print(f"⚠️ I2C 통신 오류 발생: {e}")
            v_raw, c_raw = np.nan, np.nan
            real_voltage, real_current = np.nan, np.nan
            label = 1  # 아크 감지

        data.append([now, v_raw, c_raw, real_voltage, real_current, label])
        count += 1

        # 매 100개마다 CSV로 저장
        if len(data) >= 100:
            df = pd.DataFrame(data, columns=['timestamp', 'v_raw', 'c_raw', 'voltage(V)', 'current(A)', 'label'])

            # 파일 저장
            # df.to_csv("raw_normal_dataset(860hz)_label.csv", mode='a', index=False, header=False)
            df.to_csv("raw_normal_dataset(860hz)_label_no_current.csv", mode='a', index=False, header=False)

            # 로그 출력
            print(f"[{count}] Mean Voltage: {df['voltage(V)'].mean():.2f}, Current: {df['current(A)'].mean():.2f}, Abnormal Count: {df['label'].sum()}")
            print("-" * 60)

            data = []

        time.sleep(0.001163)
except KeyboardInterrupt:
    print("🛑 실시간 감지를 종료합니다.")

[100] Mean Voltage: 6.35, Current: -18.37, Abnormal Count: 0
------------------------------------------------------------
⚠️ I2C 통신 오류 발생: [Errno 5] Input/output error
[200] Mean Voltage: 7.72, Current: -18.44, Abnormal Count: 1
------------------------------------------------------------
[300] Mean Voltage: 4.80, Current: -19.15, Abnormal Count: 0
------------------------------------------------------------
[400] Mean Voltage: 5.04, Current: -19.16, Abnormal Count: 0
------------------------------------------------------------
[500] Mean Voltage: 4.38, Current: -19.15, Abnormal Count: 0
------------------------------------------------------------
[600] Mean Voltage: 3.88, Current: -18.94, Abnormal Count: 0
------------------------------------------------------------
[700] Mean Voltage: 6.75, Current: -18.35, Abnormal Count: 0
------------------------------------------------------------
🛑 실시간 감지를 종료합니다.
